#### Day 24 - B
Swap 4 pairs of gate outputs to convert circuit into a binary adder.

#### The way this code works is the values of x and y are randomised and the last code block verifies the addition starting from z00 and moving up digit by digit. If a discrpency is found between the additon of xn + yn and zn then the code flags a gate output has been swapped at digit n. This can then be manually identified from the input.

#### I created a version of the input called 24_fix.txt with the correct gate outputs that I updated each time the code identified an issue. This is so the an earlier error won't propergate the later digit verifications and creat false positives.

#### Since the swapped gates might output the correct answer for some values of x and y, the code is ran multiple times with random x and y until an discrepency is found.

In [1]:
#Import Libraries and settings
import random as r

settings = {
    "day": "24_fix",
    "test_data": 0
}

In [2]:
def f_and(x, y):
    return x and y

def f_or(x, y):
    return x or y

def f_xor(x, y):
    return x ^ y

In [3]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    #Process inputs into dictionary
    circuit = {
        "in":{},
        "gate":{},
        "ready":[],
        "inactive":{"in":{}, "out":{}}
    }

    #Boolean to seperate the first half of the input from the second.
    initial  = True

    for line in lines:
        #If the break between the first and second second is found
        if line == "":
            initial = False
        #Process initial value for x 
        elif initial:
            wire = line.split(": ")
            wid = wire[0]
            
            wvalue = bool(int(wire[1]))
            circuit["in"][wid] = r.randint(0,1)
        else:
            #Parse line
            gate_eq = line.split(" -> ")
            gate_in = gate_eq[0].split(" ")
            wires_in = [gate_in[0], gate_in[2]]
            gate_op = gate_in[1]
            gate_out = gate_eq[1]

            #Get operator function
            if gate_op == "AND":
                func = f_and
            elif gate_op == "OR":
                func = f_or
            elif gate_op == "XOR":
                func = f_xor

            #Record gate data
            circuit["gate"][gate_out] = {
                "in": wires_in[:],
                "op": func
                }
            
            #Record gate dependencies
            for wire in wires_in:
                if wire in circuit["inactive"]["in"].keys():
                    circuit["inactive"]["in"][wire].append(gate_out)
                else:
                    circuit["inactive"]["in"][wire] = [gate_out]

            circuit["inactive"]["out"][gate_out] = wires_in

    #Process the existing state to determine inactive and ready gates
    for wire in circuit["in"].keys():
        for connection in circuit["inactive"]["in"][wire]:
            circuit["inactive"]["out"][connection].remove(wire)

            if circuit["inactive"]["out"][connection] == []:
                del circuit["inactive"]["out"][connection]
                circuit["ready"].append(connection)

        del circuit["inactive"]["in"][wire]


    return circuit

circuit = load_input(settings)

In [4]:
def apply_gate_result(gate_id, val):
    global circuit

    #Add gate to inputs
    circuit["in"][gate_id] = val

    #Update inactive tracker
    if gate_id in circuit["inactive"]["in"].keys():
        for connection in circuit["inactive"]["in"][gate_id]:
                circuit["inactive"]["out"][connection].remove(gate_id)

                #Check if the output is expecting any more inputs
                if circuit["inactive"]["out"][connection] == []:
                    #Mark the output as ready to process
                    del circuit["inactive"]["out"][connection]
                    circuit["ready"].append(connection)

        #Remove processed gate from the inactive input list
        del circuit["inactive"]["in"][gate_id]

#Process a given gate id
def process_gate(gate_id):
    global circuit

    #Load gate details
    gate = circuit["gate"][gate_id]
    #Calculate result
    res = gate["op"](circuit["in"][gate["in"][0]], circuit["in"][gate["in"][1]])

    #Update circuit with the result
    apply_gate_result(gate_id, res)

    #Debug Line
    #print(gate_id, "=>", circuit["in"][gate["in"][0]], str(gate["op"]).split(" ")[1].split("_")[1].upper(), circuit["in"][gate["in"][1]], "=", circuit["in"][gate_id])

In [5]:
#Process ready circuits
while circuit["ready"]:
    next_gate = circuit["ready"].pop(0)
    process_gate(next_gate)
    

In [10]:
#Calculate the output
z_ids = [z for z in circuit["in"].keys() if z[0] == "z"]
z_ids.sort()
val = 0

prev = "z00"
carry = circuit["in"]["x00"] and circuit["in"]["y00"]
print("00", ":", int(circuit["in"]["x00"]),"+", int(circuit["in"]["y00"]), "=", int(circuit["in"]["z00"]), ", carry:", int(carry))
for z in z_ids[1:-1]:
    
    digit = z[1:]
    x = circuit["in"]["x" + digit]
    y = circuit["in"]["y" + digit]
    if circuit["in"][z] != (x ^ y) ^ carry:
        #print(x,"XOR", y, "XOR", carry, "=", circuit["in"][z])
        print("Check digit", digit)
    print(digit, ":", int(x),"+", int(y), "+", int(carry), "=", int(circuit["in"][z]), "carry:", int(x and y))
    carry = (x and y) or (carry and (x ^ y))
    

00 : 0 + 1 = 1 , carry: 0
01 : 0 + 0 + 0 = 0 carry: 0
02 : 1 + 1 + 0 = 0 carry: 1
03 : 1 + 1 + 1 = 1 carry: 1
04 : 0 + 0 + 1 = 1 carry: 0
05 : 1 + 1 + 0 = 0 carry: 1
06 : 0 + 1 + 1 = 0 carry: 0
07 : 0 + 1 + 1 = 0 carry: 0
08 : 1 + 0 + 1 = 0 carry: 0
09 : 1 + 0 + 1 = 0 carry: 0
10 : 1 + 1 + 1 = 1 carry: 1
11 : 0 + 1 + 1 = 0 carry: 0
12 : 1 + 1 + 1 = 1 carry: 1
13 : 1 + 1 + 1 = 1 carry: 1
14 : 0 + 1 + 1 = 0 carry: 0
15 : 1 + 0 + 1 = 0 carry: 0
16 : 0 + 0 + 1 = 1 carry: 0
17 : 0 + 1 + 0 = 1 carry: 0
18 : 1 + 1 + 0 = 0 carry: 1
19 : 0 + 0 + 1 = 1 carry: 0
20 : 1 + 0 + 0 = 1 carry: 0
21 : 1 + 0 + 0 = 1 carry: 0
22 : 1 + 1 + 0 = 0 carry: 1
23 : 0 + 0 + 1 = 1 carry: 0
24 : 1 + 1 + 0 = 0 carry: 1
25 : 1 + 1 + 1 = 1 carry: 1
26 : 1 + 0 + 1 = 0 carry: 0
27 : 0 + 1 + 1 = 0 carry: 0
28 : 0 + 0 + 1 = 1 carry: 0
29 : 0 + 0 + 0 = 0 carry: 0
30 : 1 + 1 + 0 = 0 carry: 1
31 : 1 + 1 + 1 = 1 carry: 1
32 : 1 + 0 + 1 = 0 carry: 0
33 : 0 + 1 + 1 = 0 carry: 0
34 : 1 + 1 + 1 = 1 carry: 1
35 : 0 + 0 + 1 = 1 car